In [16]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohang62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [3]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [4]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)
train['context']=train['context'].apply(denoise_text)
test['context']=test['context'].apply(denoise_text)

In [5]:
train_response = []
for i in train.response.values:
    train_response.append(i.split())

test_response = []
for i in test.response.values:
    test_response.append(i.split())

train_context = []
for i in train.context.values:
    train_context.append(i.split())

test_context = []
for i in test.context.values:
    test_context.append(i.split())

In [8]:
import gensim
EMBEDDING_DIM = 200

train_response_w2vec = gensim.models.Word2Vec(sentences = train_response , size=EMBEDDING_DIM , window = 5 , min_count = 1)
train_context_w2vec = gensim.models.Word2Vec(sentences = train_context , size=EMBEDDING_DIM , window = 5 , min_count = 1)
test_response_w2vec = gensim.models.Word2Vec(sentences = test_response , size=EMBEDDING_DIM , window = 5 , min_count = 1)
test_context_w2vec = gensim.models.Word2Vec(sentences = test_context , size=EMBEDDING_DIM , window = 5 , min_count = 1)

In [6]:
train_data = np.vstack((train_response_w2vec, train_context_w2vec))
test_data = np.vstack((test_response_w2vec, test_context_w2vec))

NameError: name 'train_response_w2vec' is not defined

In [19]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=2,
args={'reprocess_input_data': True, 'overwrite_output_dir': True},use_cuda=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [10]:
import inspect
inspect.getargspec(model.train_model)

/home/rohang62/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  


ArgSpec(args=['self', 'train_df', 'multi_label', 'output_dir', 'show_running_loss', 'args', 'eval_df', 'verbose'], varargs=None, keywords='kwargs', defaults=(False, None, True, None, None, True))

In [21]:
train_2 = pd.DataFrame({
    'text': train['response'].replace(r'\n', ' ', regex=True),
    'label': train['label']
})
train_2.loc[train_2.label == "SARCASM", "label"] = 1 
train_2.loc[train_2.label != "SARCASM", "label"] = 0

In [ ]:
model.train_model(train_2)